In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random
import os

import mne
from mne.io import read_raw_ctf
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input
from keras.initializers import glorot_uniform
from keras import backend as K
from keras import optimizers
from keras.utils.vis_utils import plot_model
from load_dataset import *

Using TensorFlow backend.


In [34]:
train_x_orig, test_x_orig, train_y_orig, test_y_orig = load_dataset(window_size=0.2, test_size=0.33, random_state=0)

Start loading data...
From dataset: anonepi_02.ds
ds directory : /Users/mlion0200/GitRepo/bcni_epi/notebook/../data/anonepi_02.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       0.31   79.57   -0.00 mm <->    0.31   79.57    0.00 mm (orig :  -47.56   66.09 -265.98 mm) diff =    0.000 mm
      -0.31  -79.57   -0.00 mm <->   -0.31  -79.57    0.00 mm (orig :   56.93  -53.61 -274.89 mm) diff =    0.000 mm
     103.93   -0.00    0.00 mm <->  103.93   -0.00    0.00 mm (orig :   78.68   68.54 -232.42 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
Picked positions of 21 EEG channels from channel info
    21 EEG locations added to Polhemus data.
    Measurement info composed.
Finding samples for /Users/mlion0200/GitRepo/bcni_epi/notebook/../data/anonepi_02.ds/anonepi_02.meg4: 
    System

In [35]:
train_x_array = np.asarray(train_x_orig)
test_x_array = np.asarray(test_x_orig)
train_y_array = np.asarray(train_y_orig)
test_y_array = np.asarray(test_y_orig)
m_train = train_x_array.shape[0]
num_px = train_x_array.shape[1]
m_test = test_x_array.shape[0]

print ("Number of training examples: " + str(m_train))
print ("Number of testing examples: " + str(m_test))
print ("train_x_orig shape: " + str(train_x_array.shape))
print ("train_y shape: " + str(train_y_array.shape))
print ("test_x_orig shape: " + str(test_x_array.shape))
print ("test_y shape: " + str(test_y_array.shape))

Number of training examples: 64
Number of testing examples: 32
train_x_orig shape: (64, 151, 121)
train_y shape: (64,)
test_x_orig shape: (32, 151, 121)
test_y shape: (32,)


In [57]:
def norm_and_reshape(train_x, test_x):
    train_x_norm = train_x_array.reshape(train_x_array.shape[0], train_x_array.shape[1] * train_x_array.shape[2])
    test_x_norm = test_x_array.reshape(test_x_array.shape[0], test_x_array.shape[1] * test_x_array.shape[2])
    train_x = normalize(train_x_norm, norm='l2')
    test_x = normalize(test_x_norm, norm='l2')
    return train_x, test_x

In [58]:
train_x, test_x = norm_and_reshape(train_x_array, test_x_array)

In [50]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [51]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [52]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [ ]:
for name, clf in zip(names, classifiers):
    clf.fit(train_x, train_y_array)
    score = clf.score(test_x, test_y_array)
    print(name, 'Score: ', score)

Nearest Neighbors Score:  0.84375
Linear SVM Score:  0.4375
RBF SVM Score:  0.84375
Gaussian Process Score:  0.875
Decision Tree Score:  0.78125
Random Forest Score:  0.5625


/Users/mlion0200/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Neural Net Score:  0.875
